# Genetic determinants of incident CH

In [1]:
library(data.table)

In [5]:
setwd("/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/rg2022/")

In [10]:
# load prev. CH GWAS loci 
chip_hg38tohg19 <- fread("ch_rg22.hg19.csv", header=T)
str(chip_hg38tohg19)  

dnmt3a_hg38tohg19 <- fread("dnmt3a_rg22.hg19.csv", header=T)
str(dnmt3a_hg38tohg19)

tet2_hg38tohg19 <- fread("tet2_rg22.hg19.csv", header=T)
str(tet2_hg38tohg19)

Classes 'data.table' and 'data.frame':	57 obs. of  51 variables:
 $ varID            : chr  "1:226374920:A:C" "10:103908421:C:CTT" "11:108043157:T:C" "11:108435509:C:A" ...
 $ chr_hg19         : int  1 10 11 11 11 12 14 17 17 17 ...
 $ POS_hg19         : int  226562621 105668179 107913884 108306236 108315606 26589770 101178715 55463857 7571752 7731041 ...
 $ locus            : chr  "1-1" "10-1" "11-1" "11-1" ...
 $ Chr              : int  1 10 11 11 11 12 14 17 17 17 ...
 $ Pos              : int  226374920 103908421 108043157 108435509 108444879 26436837 100712378 57386496 7668434 7827723 ...
 $ Ref              : chr  "A" "C" "T" "C" ...
 $ Alt              : chr  "C" "CTT" "C" "A" ...
 $ Trait            : chr  "CHIP_inclusive" "CHIP_inclusive" "CHIP_inclusive" "CHIP_inclusive" ...
 $ Model            : chr  "ADD-WGR-FIRTH" "ADD-WGR-FIRTH" "ADD-WGR-FIRTH" "ADD-WGR-FIRTH" ...
 $ Effect_UKB       : num  1.115 0.914 0.921 1.12 1.228 ...
 $ LCI_Effect_UKB   : num  1.086 0.887 0.896 1.09

In [33]:
## Load phenotypes
aric_baseline_n_v05 <- fread("../../pheno/aric_baseline_n_v05_N4189.pheno_ch_status_trajectory.23Mar2023.csv", header=T)
aric_baseline_n_v05$dAge <- aric_baseline_n_v05$Age - aric_baseline_n_v05$age_base

In [34]:
# dbGaP linker
aric_dbgap_linker <- fread("../../pheno/phs000280.v7.pht001441.v7.p1.ARIC_Sample.MULTI.txt.gz", 
                           skip = 10, fill = TRUE, header = T)

aric_dbgap_linker <- aric_dbgap_linker[,c(1,2,4,5)]

aric_dbgap_linker <- subset(aric_dbgap_linker, !duplicated(aric_dbgap_linker$SAMPLE_ID))

table(aric_baseline_n_v05$GWAS_ID %in% aric_dbgap_linker$SAMPLE_ID, exclude = NULL)



TRUE 
4189 

In [35]:
## link dbGaP ids with GWASID
aric_baseline_n_v05 <- merge(aric_dbgap_linker, 
                              aric_baseline_n_v05,
                              by.x = "SAMPLE_ID",
                              by.y = "GWAS_ID")

aric_baseline_n_v05$SUBJECT_ID <- as.numeric(aric_baseline_n_v05$SUBJECT_ID)

head(aric_baseline_n_v05)

SAMPLE_ID,dbGaP_Subject_ID,dbGaP_Sample_ID,SUBJECT_ID,ARIC_ID,GWASID_Visit,cram_id,Visit.x,Gender,Age,...,CH_SF,CH_DDR,CH_DNMT3A,CH_TET2,CH_ASXL1,nonHDL_base_std,age_base_sqr,Center,incident_CH_or_growingClones,dAge
<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,...,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<chr>,<int>,<int>
A00004,288368,1786858,131031724,M102321,A00004_V05,H5TH7DSX2-2-IDUDI0072,V05,F,71,...,NA,NA,1,NA,NA,-0.06472577,2601,M,1,20
A00009,227213,1786861,131051876,F101210,A00009_V05,HNTCYDSX2-2-IDUDI0060,V05,F,76,...,0,0,0,0,0,-0.16103907,3249,F_J,0,19
A00012,232161,1786863,131188895,W121907,A00012_V05,HNTGWDSX2-1-IDUDI0006,V05,F,79,...,0,0,0,0,0,-0.16966414,3249,W,0,22
A00013,222837,1387960,131123798,M263966,A00013_V05,HKNWTDSX2-3-IDUDI0063,V05,M,82,...,NA,NA,NA,1,NA,-0.07766338,3844,M,1,20
A00015,228166,1786865,131154692,F101837,A00015_V05,H252LDSX3-1-IDUDI0036,V05,M,82,...,0,0,0,0,0,-0.13947639,3969,F_J,0,19
A00017,224264,1786866,131017525,M132251,A00017_V05,HY2KGDSX2-1-IDUDI0021,V05,M,83,...,0,0,0,0,0,-0.07047582,3721,M,0,22


In [36]:
## PCA data
pca_ea <- fread("../../pca/ARIC_EurAm_chr1_22.aray_snps.eigenvec", header = T)
# pca_ea$IID <- as.character(pca_ea$IID)
pca_aa <- fread("../../pca/ARIC_AfrAm_chr1_22.aray_snps.eigenvec", header = T)
# pca_ea$IID <- as.character(pca_ea$IID)
table(aric_baseline_n_v05$SUBJECT_ID %in% pca_ea$IID)


FALSE  TRUE 
 1497  2692 

In [37]:
# EA: 2692
aric_baseline_n_v05_ea <- merge(aric_baseline_n_v05, pca_ea, 
                                by.x="SUBJECT_ID", by.y="IID")

nrow(aric_baseline_n_v05_ea)

[1] 2692

In [38]:
# EA: 2692
aric_baseline_n_v05_aa <- merge(aric_baseline_n_v05, pca_aa, 
                                by.x="SUBJECT_ID", by.y="IID")

nrow(aric_baseline_n_v05_aa)

[1] 701

In [50]:
names(aric_baseline_n_v05_ea[, c(1,1,2,6,3:5,55,9:21,109, 61,107,108,112:121,85,86,87,89)])
aric_baseline_n_v05_ea_gwas <- aric_baseline_n_v05_ea[, c(1,1,2,6,3:5,55,9:21,109, 61,107,108,112:121,85,86,87,89)]
names(aric_baseline_n_v05_ea_gwas) <- c("FID","IID","GWAS_ID", names(aric_baseline_n_v05_ea_gwas)[4:39])
str(aric_baseline_n_v05_ea_gwas)

[1] "SUBJECT_ID"                   "SUBJECT_ID"                  
 [3] "SAMPLE_ID"                    "GWASID_Visit"                
 [5] "dbGaP_Subject_ID"             "dbGaP_Sample_ID"             
 [7] "ARIC_ID"                      "Visit.y"                     
 [9] "Gender"                       "Age"                         
[11] "CH_baseline"                  "CH_v05"                      
[13] "incident_CH"                  "incident_CHvaf05"            
[15] "incident_CHvaf10"             "incident_DNMT3A"             
[17] "incident_TET2"                "incident_ASXL1"              
[19] "incident_DTA"                 "incident_SF"                 
[21] "incident_DDR"                 "incident_CH_or_growingClones"
[23] "age_base"                     "age_base_sqr"                
[25] "Center"                       "PC1"                         
[27] "PC2"                          "PC3"                         
[29] "PC4"                          "PC5"                         
[31] "PC6"                          "PC7"                         
[33] "PC8"                          "PC9"                         
[35] "PC10"                         "ever_smoke"                  
[37] "v2_vs_other"                  "Sex"                         
[39] "BMI_cat"

Classes 'data.table' and 'data.frame':	2692 obs. of  39 variables:
 $ FID                         : int  131000113 131000147 131000179 131000253 131000354 131000401 131000455 131000596 131000641 131000672 ...
 $ IID                         : int  131000113 131000147 131000179 131000253 131000354 131000401 131000455 131000596 131000641 131000672 ...
 $ GWAS_ID                     : chr  "A15057" "A04826" "A02787" "A07971" ...
 $ GWASID_Visit                : chr  "A15057_V05" "A04826_V05" "A02787_V05" "A07971_V05" ...
 $ dbGaP_Subject_ID            : int  228789 232712 230453 231733 224837 229698 234835 230455 228925 229174 ...
 $ dbGaP_Sample_ID             : int  1336383 1788922 1788029 1389147 1787805 1388504 1787033 1388831 1336156 1334770 ...
 $ ARIC_ID                     : chr  "W293092" "M127886" "W235739" "M288311" ...
 $ Visit.y                     : chr  "V02" "V02" "V02" "V02" ...
 $ Gender                      : chr  "M" "M" "F" "M" ...
 $ Age                         : int 

In [51]:
names(aric_baseline_n_v05_aa[, c(1,1,2,6,3:5,55,9:21,109, 61,107,108,112:121,85,86,87,89)])
aric_baseline_n_v05_aa_gwas <- aric_baseline_n_v05_aa[, c(1,1,2,6,3:5,55,9:21,109, 61,107,108,112:121,85,86,87,89)]
names(aric_baseline_n_v05_aa_gwas) <- c("FID","IID","GWAS_ID", names(aric_baseline_n_v05_aa_gwas)[4:39])
str(aric_baseline_n_v05_aa_gwas)

[1] "SUBJECT_ID"                   "SUBJECT_ID"                  
 [3] "SAMPLE_ID"                    "GWASID_Visit"                
 [5] "dbGaP_Subject_ID"             "dbGaP_Sample_ID"             
 [7] "ARIC_ID"                      "Visit.y"                     
 [9] "Gender"                       "Age"                         
[11] "CH_baseline"                  "CH_v05"                      
[13] "incident_CH"                  "incident_CHvaf05"            
[15] "incident_CHvaf10"             "incident_DNMT3A"             
[17] "incident_TET2"                "incident_ASXL1"              
[19] "incident_DTA"                 "incident_SF"                 
[21] "incident_DDR"                 "incident_CH_or_growingClones"
[23] "age_base"                     "age_base_sqr"                
[25] "Center"                       "PC1"                         
[27] "PC2"                          "PC3"                         
[29] "PC4"                          "PC5"                         
[31] "PC6"                          "PC7"                         
[33] "PC8"                          "PC9"                         
[35] "PC10"                         "ever_smoke"                  
[37] "v2_vs_other"                  "Sex"                         
[39] "BMI_cat"

Classes 'data.table' and 'data.frame':	701 obs. of  39 variables:
 $ FID                         : int  131000053 131000094 131000655 131001064 131001173 131001245 131001306 131001309 131001392 131001460 ...
 $ IID                         : int  131000053 131000094 131000655 131001064 131001173 131001245 131001306 131001309 131001392 131001460 ...
 $ GWAS_ID                     : chr  "A05162" "A14961" "A12667" "A11283" ...
 $ GWASID_Visit                : chr  "A05162_V05" "A14961_V05" "A12667_V05" "A11283_V05" ...
 $ dbGaP_Subject_ID            : int  230296 229859 224585 228563 229539 224645 226634 232824 229013 231474 ...
 $ dbGaP_Sample_ID             : int  699030 701005 1792295 1389627 699670 699409 699581 701049 699835 698355 ...
 $ ARIC_ID                     : chr  "J187560" "J124491" "J307167" "J304595" ...
 $ Visit.y                     : chr  "V02" "V02" "V01" "V02" ...
 $ Gender                      : chr  "M" "F" "F" "M" ...
 $ Age                         : int  72 69 67

In [52]:
## GWAS_ID==SAMPLE_ID
# fwrite(aric_baseline_n_v05_ea, "../aric_baseline_n_v05_ea_PCA.2023Apr3.csv", 
  #      row.names = F, col.names = T, sep=",", na="NA")
# fwrite(aric_baseline_n_v05_ea_gwas, "../aric_baseline_n_v05_ea_PCA_GWAS.2023Apr3.tsv",
  #      row.names = F, col.names = T, sep="\t", na="NA", quote = F)
# 
# fwrite(aric_baseline_n_v05_aa, "../aric_baseline_n_v05_aa_PCA.2023Apr3.csv", 
  #      row.names = F, col.names = T, sep=",", na="NA")
# fwrite(aric_baseline_n_v05_aa_gwas, "../aric_baseline_n_v05_aa_PCA_GWAS.2023Apr3.tsv",
  #      row.names = F, col.names = T, sep="\t", na="NA", quote = F)
